In [53]:
import requests
import pandas as pd
import json

In [ ]:
response = requests.get("https://api.mfapi.in/mf", allow_redirects = True)
data = json.loads(response.content)
df_mf_list = pd.json_normalize(data)
# df_mf_list.to_csv('mf_list.csv')

In [ ]:
mf_data_response = requests.get("https://api.mfapi.in/mf/120843", allow_redirects = True)
mf_data_1 = json.loads(mf_data_response.content)
# mf_data = pd.json_normalize(mf_data_1['data'])
mf_data = pd.json_normalize(mf_data_1)
mf_data_df = pd.DataFrame.from_dict(mf_data['data'][0])
mf_data_df['date'] = pd.to_datetime(mf_data_df['date'], format = '%d-%m-%Y')